In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = "ollama simple app"
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [13]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
llm = ChatGroq(model="llama-3.3-70b-versatile")

In [20]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "Translate this text into {language} you do not have to specify any kind of information in english just translate "),
    ("human", "{input}")
])

In [21]:
parser = StrOutputParser()

In [22]:
chain = prompt | llm | parser

In [23]:
res = chain.invoke({"language":"japanese", "input":"hello my name is anuj"})
print(res)

こんにちは、私の名前はアヌジュです


## simple GenAI app that can remember with sessions

In [24]:
from langchain_core.messages import HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

store = {} # this stores all the session id

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [25]:
with_message_history = RunnableWithMessageHistory(llm,get_session_history)

In [26]:
config = {"configurable":{"session_id":"chat1"}}

In [27]:
with_message_history.invoke(
    [HumanMessage(content="Hi, I am anuj. i am currently learning gen AI"),
     ],config=config
)

AIMessage(content="Hello Anuj, nice to meet you. That's exciting to hear that you're learning about General AI (gen AI). General AI refers to a type of artificial intelligence that can perform any intellectual task that a human can. It's a vast and complex field that has the potential to revolutionize many areas of our lives.\n\nWhat specific aspects of gen AI are you currently learning about? Are you exploring topics like machine learning, natural language processing, computer vision, or something else? I'm here to help and provide guidance if you have any questions or need clarification on any concepts.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 117, 'prompt_tokens': 48, 'total_tokens': 165, 'completion_time': 0.316511145, 'prompt_time': 0.002354651, 'queue_time': 0.051647039, 'total_time': 0.318865796}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs':

In [29]:
with_message_history.invoke(
    [HumanMessage(content="what is my name? and do you know what i am learning right now?"),
     ],config=config
)

AIMessage(content='Your name is Anuj, and you are currently learning about General AI (gen AI).', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 211, 'total_tokens': 230, 'completion_time': 0.039471172, 'prompt_time': 0.0113971, 'queue_time': 0.051461044000000004, 'total_time': 0.050868272}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--761c5615-1e4f-4481-8260-4d5da6d48162-0', usage_metadata={'input_tokens': 211, 'output_tokens': 19, 'total_tokens': 230})

In [30]:
config_2 = {"configurable":{"session_id":"chat2"}}
with_message_history.invoke(
    [HumanMessage(content="Hi, do you know my name?"),
     ],config=config_2
)

AIMessage(content="I don't have any information about your name. I'm a large language model, I don't have the ability to retain information about individual users or conversations. Each time you interact with me, it's a new conversation and I don't have any prior knowledge about you. Would you like to introduce yourself?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 43, 'total_tokens': 106, 'completion_time': 0.12760372, 'prompt_time': 0.007024508, 'queue_time': 0.052100562, 'total_time': 0.134628228}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9a8b91ba77', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--e0393058-e67a-46fe-9aa0-c189074bebee-0', usage_metadata={'input_tokens': 43, 'output_tokens': 63, 'total_tokens': 106})